In [24]:
from bs4 import BeautifulSoup
import requests

def generate_url(page: int) -> str:
    return f"https://freedom.pl/mieszkania/kolobrzeg/zp/page/{page}/"

def get_soup(url: str) -> BeautifulSoup:
    response = requests.get(url)
    return BeautifulSoup(response.content, "html.parser")

In [18]:
flats_urls = set()

for page in range(1, 5):
    url = generate_url(page)
    soup = get_soup(url)

    flats = soup.find_all("div", class_="offert")
    for flat in flats:
        url = flat.find("a")["href"]
        flats_urls.add(url)

In [69]:
import pandas as pd

df1 = pd.DataFrame(flats_urls)
df1.to_csv('./data/flats_urls.csv', index=False, header=False)
df1

,0
0,https://freedom.pl/oferta/mieszkanie-na-sprzed...
1,https://freedom.pl/oferta/mieszkanie-na-sprzed...
2,https://freedom.pl/oferta/mieszkanie-na-sprzed...
3,https://freedom.pl/oferta/mieszkanie-na-sprzed...
4,https://freedom.pl/oferta/mieszkanie-na-sprzed...
...,...
65,https://freedom.pl/oferta/mieszkanie-na-sprzed...
66,https://freedom.pl/oferta/mieszkanie-na-sprzed...
67,https://freedom.pl/oferta/mieszkanie-na-sprzed...
68,https://freedom.pl/oferta/mieszkanie-na-sprzed...


In [70]:
def format_text(text: str) -> str:
    # text = text.lower()
    text = text.strip()
    text = text.replace("\n", "")
    text = text.replace(":", "")
    return text


def fetch_flat(row):
    url = row[0]
    soup = get_soup(url)
    details = soup.find_all("div", class_="detail")

    offer_table = {
        "url": url,
    }

    for detail in details:
        fields = detail.find_all("li")
        for field in fields:
            key = field.find("strong").text
            key = format_text(key)

            value = field.find("small").text
            value = format_text(value)

            offer_table[key] = value

    offer_table["opis"] = soup.find("div", class_="desc-tab").text.strip()

    return offer_table


df2 = df1.apply(fetch_flat, axis=1, result_type="expand")
df2.to_csv('./data/flats_dirty.csv', index=False)
df2

,url,Rynek,Cena,Powierzchnia,Cena za m2,Liczba pokoi,Piętro,Liczba pięter,Czynsz administracyjny,Rodzaj nieruchomości,...,Liczba oddzielnych toalet,Powierzchnie pokoi,Powierzchnia przedpokoju,Powierzchnia kuchni,Cena za garaż/Miejsce parkingowe,Zobacz spacer 360°,Stan mieszkania,Ciepła woda,Rodzaj ogrzewania,Typ okien
0,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,450 000 zł,38.66 m2,11 640 zł,2,0,3,443.28,Mieszkanie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,520 000 zł,32.92 m2,15 796 zł,1,0,7,NaN,Mieszkanie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,690 000 zł,71.87 m2,9 601 zł,3,3,4,436,Mieszkanie,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,250 000 zł,33.29 m2,7 510 zł,1,0,2,332.91,Mieszkanie,...,NaN,"17,20",3.2,"9,00 m2",0 zł,NaN,NaN,NaN,NaN,NaN
4,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,798 000 zł,85.26 m2,9 360 zł,3,3,3,700,Mieszkanie,...,NaN,NaN,NaN,NaN,80 000 zł,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,830 000 zł,67.7 m2,12 260 zł,3,3,4,800,Mieszkanie,...,NaN,NaN,6.4,"11,6 m2",100 000 zł,https//my.matterport.com/show/?m=Wqa2FqdGUsH,NaN,NaN,NaN,NaN
66,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,880 000 zł,52.34 m2,16 813 zł,2,4,5,770,Mieszkanie,...,NaN,NaN,NaN,NaN,NaN,https//mpembed.com/show/?m=xnA1mhEH6dk&mpu=862,NaN,NaN,NaN,NaN
67,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,900 000 zł,48.47 m2,18 568 zł,2,3,4,1064,Mieszkanie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,400 000 zł,43.2 m2,9 259 zł,2,4,4,710,Mieszkanie,...,NaN,"20,11",3.4,"3,8 m2",NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
df3 = df2[["url", "Numer oferty"]]
df3

,url,Numer oferty
0,https://freedom.pl/oferta/mieszkanie-na-sprzed...,29361/3685/OMS
1,https://freedom.pl/oferta/mieszkanie-na-sprzed...,29044/3685/OMS
2,https://freedom.pl/oferta/mieszkanie-na-sprzed...,21357/3685/OMS
3,https://freedom.pl/oferta/mieszkanie-na-sprzed...,29076/3685/OMS
4,https://freedom.pl/oferta/mieszkanie-na-sprzed...,29623/3685/OMS
...,...,...
65,https://freedom.pl/oferta/mieszkanie-na-sprzed...,28566/3685/OMS
66,https://freedom.pl/oferta/mieszkanie-na-sprzed...,24981/3685/OMS
67,https://freedom.pl/oferta/mieszkanie-na-sprzed...,26529/3685/OMS
68,https://freedom.pl/oferta/mieszkanie-na-sprzed...,28902/3685/OMS


In [94]:
import os

def download_photos(row):
    nr_oferty = row["Numer oferty"]
    url = row["url"]

    soup = get_soup(url)

    images = soup.find("div", class_="swiper-container").find_all("img")
    for image in images:
        src = image["src"].split("-")[:-1]
        photo_url = "-".join(src) + ".jpg"

        filename = photo_url.split("/")[-1]

        directory = f"./images/{nr_oferty}"
        os.makedirs(directory, exist_ok=True)

        response = requests.get(photo_url)
        with open(f"{directory}/{filename}", 'wb') as file:
            file.write(response.content)

    return row

df3.apply(download_photos, axis=1)

,url,Numer oferty
0,https://freedom.pl/oferta/mieszkanie-na-sprzed...,29361/3685/OMS
1,https://freedom.pl/oferta/mieszkanie-na-sprzed...,29044/3685/OMS
2,https://freedom.pl/oferta/mieszkanie-na-sprzed...,21357/3685/OMS
3,https://freedom.pl/oferta/mieszkanie-na-sprzed...,29076/3685/OMS
4,https://freedom.pl/oferta/mieszkanie-na-sprzed...,29623/3685/OMS
...,...,...
65,https://freedom.pl/oferta/mieszkanie-na-sprzed...,28566/3685/OMS
66,https://freedom.pl/oferta/mieszkanie-na-sprzed...,24981/3685/OMS
67,https://freedom.pl/oferta/mieszkanie-na-sprzed...,26529/3685/OMS
68,https://freedom.pl/oferta/mieszkanie-na-sprzed...,28902/3685/OMS
